<a href="https://colab.research.google.com/github/cctbx/cctbx_tutorials/blob/main/struktura_2024/Struktura_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction to CCTBX**

The CCTBX project (computational crystallography toolbox) initially started as a library containing algorithms for the handling of unit cells, space groups, and atomic scatterers. Since then it has grown into a large project to support a variety of structural biology workflows, including x-ray and neutron crystallography, cryo-electron microscopy, and x-ray free electron lasers.

The CCTBX library is written in Python, with many of the lower level algorithms accelerated using C++ via the Boost Python bindings.

The principal high-level cctbx objects are:

*   Data Manager: Read and write files and keep track of them
*   Model Manager: A molecular model or models and restraints that go with them
*   Map Manager: A 3D map or maps and information about gridding and map symmetry
*   Map Model Manager: One or more related maps and models as a group
*   F-Model Manager: Combines a model and a related Miller array (map coefficeints, or other data for each index of a map)

These high-level cctbx objects are composed of lower-level objects that implement various concepts in macromolecular crystallography, which are in turn composed of CCTBX flex arrays for efficient computation of numerical data. Finally, user defined parameters are managed and shared betwen objects using the CCTBX Phil module.

The examples presented here aim to illustrate working with the core CCTBX data types using example data. Much of the content is adapted from the official CCTBX documentation: http://cci.lbl.gov/docs/cctbx/

# **Setting up cctbx**
First we need to set up the cctbx in this colab. Execute the following cells to install condacolab.



In [ ]:
#@title Install condacolab so that conda packages can be used

# Install condacolab so that conda packages can be used
# https://github.com/conda-incubator/condacolab
#
# Important notes
# https://github.com/conda-incubator/condacolab#shortcomings
# 1) The kernel will automatically be restarted and show an error message about
#    a crash ("Your session crashed for an unknown reason").
#    The error can be ignored.
# 2) Only the "base" environment is available, so do not create a separate
#    environment for packages.
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
#@title Install cctbx-base
# Wait until the error message appears near the bottom of the notebook before
# proceeding.
!conda install -q cctbx-base

# conda installs ${CONDA_PREFIX}/share/cctbx into /usr/local instead of /usr
# Make a copy to avoid errors
!cp -af /usr/local/share/cctbx /usr/share/

# sys.path does not have some directories with libraries, so add them here
import sys
py_ver = f'{sys.version_info.major}.{sys.version_info.minor}'
for d in [f'/usr/local/lib/python{py_ver}/lib-dynload', '/usr/local/lib']:
  if d not in sys.path:
    sys.path.insert(0, d)

# final check
import os
if os.path.isdir('/usr/local/share/cctbx') \
  and f'/usr/local/lib/python{py_ver}/lib-dynload' in sys.path \
  and '/usr/local/lib' in sys.path:
  print('Finished installing cctbx-base')
else:
  raise RuntimeError('There was an error fixing up the installation of cctbx-base')


Channels:
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

# All requested packages already installed.

Finished installing cctbx-base


# **Test installation**

Let's check if everything worked by running some cctbx code.

We'll create 100 random numbers in a flex array.

In [ ]:
import cctbx
from scitbx.array_family import flex
a = flex.random_double(100)
print('Smallest value among the 100 values: ', flex.min(a))
print('Size of the array: ', a.size())

Smallest value among the 100 values:  0.021549243146825492
Size of the array:  100



Thanks to Georg Mlynek for this example!


In [ ]:
from cctbx import crystal, miller

crystal_symmetry = crystal.symmetry((10, 20, 30, 90, 90, 90), 'P212121')
ms = miller.build_set(
    crystal_symmetry=crystal_symmetry,
    anomalous_flag=True,
    d_min=2,
    d_max=10
)
crystal_symmetry.show_summary()
print('Resolution range', ms.d_max_min())
print('Number of reflections', ms.indices().size())

Unit cell: (10, 20, 30, 90, 90, 90)
Space group: P 21 21 21 (No. 19)
Resolution range (10.0, 2.0)
Number of reflections 762


# **File IO with the DataManager**

When you want to perform some actions on data or a model, you have to start with reading the file. Once the action is done, you probably want to write the files.

To do this conveniently, you can use the cctbx DataManager. The DataManager lets you read and write files describing atomic models, restraints, reflection data files, symmetry files, sequence files, and MRC/CCP4 format map files.

In [ ]:
from libtbx.program_template import ProgramTemplate
from libtbx.utils import Sorry

class TestProgram(ProgramTemplate):
  program_name = "new_program"
  description = """
This is a demo of my new program
"""
  master_phil_str = """
general_param = None
  .type = str
  .help = A general parameter that is a string
bool_param = None
  .type = bool
  .help = Something that is True or False
parameter_section {
  specific_param_1 = 0
    .type = int
    .help = This is an example of a numerical parameter in a PHIL section
  specific_param_2 = 3.14
    .type = float
    .help = This is another example
}
"""
  def validate(self):
    if self.params.parameter_section.specific_param_1 < 0:
      raise Sorry("The specific_param_1 must be 0 or greater")

  def run(self):
    # just add some numbers
    self.results = self.params.parameter_section.specific_param_1 + self.params.parameter_section.specific_param_2
    self._print(self.results)

  def get_results(self):
    return self.results

In [ ]:
from iotbx.cli_parser import run_program

#run_program(program_class=TestProgram, args=[])
run_program(program_class=TestProgram, args=['bool_param=False', 'param_1=10'])

Starting new_program
on Tue Sep 10 18:21:46 2024 by root

Processing files:
-------------------------------------------------------------------------------

  No files found

Processing PHIL parameters:
-------------------------------------------------------------------------------

  Adding command-line PHIL:
  -------------------------
    bool_param=False
    param_1=10

Final processed PHIL parameters:
-------------------------------------------------------------------------------
  bool_param = False
  parameter_section {
    specific_param_1 = 10
  }


Starting job
13.14

Job complete
usr+sys time: 17.32 seconds
wall clock time: 17.35 seconds


13.14